In [22]:
# Import Libs
import io
import base64
from folium.plugins import FastMarkerCluster, HeatMap
import folium
import pandas as pd
import streamlit as st

# read data
# df_pararius
df_pararius = pd.read_csv('app/df_coo_pararius.csv', index_col=[0])

In [24]:
# organize filter
filter_ = \
    (df_pararius['interior'].isin(['Furnished'])) & \
    (df_pararius['status'].isin(['Free'])) & \
    (df_pararius['price'] >= 10) & \
    (df_pararius['price'] <= 2000) & \
    (df_pararius['surface-area'] >= 0) & \
    (df_pararius['surface-area'] <= 1000) & \
    (df_pararius['number-of-rooms'] >= 0) & \
    (df_pararius['number-of-rooms'] <= 1000)
df_pararius[filter_]

,price,address,street,agency,irl,image,surface-area,status,number-of-rooms,interior,...,Gemeente,Provincie,area,neighborhood,city,url,link,img,train,deal
3,1350.0,1083 TL Amsterdam (Buitenveldert-Oost),Apartment Oldengaarde,Expat Rentals Nederland,/apartment-for-rent/amsterdam/1ca860df/oldenga...,https://media.pararius.nl/image/PR0001659000/P...,40,Free,2,Furnished,...,Amsterdam,Noord-Holland,Buitenveldert-Oost,Buitenveldert Oost Midden,Amsterdam,https://www.pararius.com/apartment-for-rent/am...,<a target='_blank' href=https://www.pararius.c...,<a href=https://www.pararius.com/apartment-for...,0.009200,0.031111
4,1450.0,2583 CA Den Haag (Vissershaven),Apartment Datheenstraat,Infinitas Real Estate,/apartment-for-rent/den-haag/bfb90e2a/datheens...,https://casco-media-prod.global.ssl.fastly.net...,71,Free,3,Furnished,...,'s-Gravenhage,Zuid-Holland,Wijk 07 Scheveningen,Vissershaven,Den Haag,https://www.pararius.com/apartment-for-rent/de...,<a target='_blank' href=https://www.pararius.c...,<a href=https://www.pararius.com/apartment-for...,0.043865,0.051033
5,750.0,1641 LX Spierdijk (Spierdijkerweg en omgeving),Room Spierdijkerweg,Expat Rentals Nederland,/room-for-rent/spierdijk/9358cda5/spierdijkerweg,https://media.pararius.nl/image/PR0001659000/P...,10,Free,1,Furnished,...,Koggenland,Noord-Holland,Wijk 01 Spierdijk,Spierdijk,Spierdijk,https://www.pararius.com/room-for-rent/spierdi...,<a target='_blank' href=https://www.pararius.c...,<a href=https://www.pararius.com/room-for-rent...,0.045122,0.014666
8,775.0,3022 AC Rotterdam (Nieuwe Westen),Apartment Vierambachtsstraat,Dop & Dop Consultancy,/apartment-for-rent/rotterdam/84e43b1d/vieramb...,https://casco-media-prod.global.ssl.fastly.net...,25,Free,1,Furnished,...,Rotterdam,Zuid-Holland,Delfshaven,Nieuwe Westen,Rotterdam,https://www.pararius.com/apartment-for-rent/ro...,<a target='_blank' href=https://www.pararius.c...,<a href=https://www.pararius.com/apartment-for...,0.010280,0.033548
10,1400.0,2015 AM Haarlem (Bloemenbuurt),Apartment Narcisplantsoen 104,Van Oosterom Makelaar,/apartment-for-rent/haarlem/264435aa/narcispla...,https://casco-media-prod.global.ssl.fastly.net...,74,Free,3,Furnished,...,Haarlem,Noord-Holland,Duinwijk,Bloemenbuurt,Haarlem,https://www.pararius.com/apartment-for-rent/ha...,<a target='_blank' href=https://www.pararius.c...,<a href=https://www.pararius.com/apartment-for...,0.010776,0.055000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5343,1650.0,"3531 VL Utrecht (Laan van Nieuw-Guinea, Spinoz...",House Makassarstraat 75,The Brick Club,/house-for-rent/utrecht/8c6167fe/makassarstraat,https://casco-media-prod.global.ssl.fastly.net...,116,Free,5,Furnished,...,Utrecht,Utrecht,Wijk 01 West,"Laan van Nieuw-Guinea, Spinozaweg e.o.",Utrecht,https://www.pararius.com/house-for-rent/utrech...,<a target='_blank' href=https://www.pararius.c...,<a href=https://www.pararius.com/house-for-ren...,0.011423,0.073333
5371,1900.0,1016 ZA Amsterdam (Jordaan),Apartment Passeerdersstraat 124,Amsterdam Wonen,/apartment-for-rent/amsterdam/729f8cb3/passeer...,https://casco-media-prod.global.ssl.fastly.net...,67,Free,3,Furnished,...,Amsterdam,Noord-Holland,Jordaan,Passeerdersgrachtbuurt,Amsterdam,https://www.pararius.com/apartment-for-rent/am...,<a target='_blank' href=https://www.pararius.c...,<a href=https://www.pararius.com/apartment-for...,0.022714,0.036842
5391,1850.0,2517 AP Den Haag (Zeeheldenkwartier),Apartment Laan van Meerdervoort 84 26.,Estata Makelaars O.G.,/apartment-for-rent/den-haag/f501130b/laan-van...,https://casco-media-prod.global.ssl.fastly.net...,85,Free,3,Furnished,...,'s-Gravenhage,Zuid-Holland,Wijk 22 Zeeheldenkwartier,Zeeheldenkwartier,Den Haag,https://www.pararius.com/apartment-for-rent/de...,<a target='_blank' href=https://www.pararius.c...,<a href=https://www.pararius.com/apartment-for...,0.015076,0.047567
5434,1200.0,2586 KT Den Haag (Scheveningen Badplaats),Apartment Vierloper,Sonar appartementen,/apartment-for-ren

In [ ]:


# yelp
#yelp = pd.read_csv('yelp.csv', index_col=[0])

# create streamlit page
st.set_page_config(layout="wide")
# config streamlit layout
hide_streamlit_style = """
<style>
#MainMenu {visibility: hidden;}
footer {visibility: hidden;}
.css-1y0tads {padding-top: 0rem;}
.css-hby737 {padding: 1rem 1rem;}
.css-ijjfg8 {width: 25px;}
</style
"""
st.markdown(hide_streamlit_style, unsafe_allow_html=True)
st.title("Places to rent in The Netherlands")

# create filter on sidebar
city = st.sidebar.multiselect("city", options=list(
    df_pararius['city'].unique()), default=['Den Haag'])

# apply filter
df_pararius = df_pararius[df_pararius['city'].isin(city)]

# Filter for price
max_price = int(df_pararius['price'].max())
min_price = int(df_pararius['price'].min())
col1, col2 = st.sidebar.columns(2)
price_selected_0 = col1.number_input(
    'Price (Min)', min_value=0, max_value=max_price, value=0, step=50)
price_selected_1 = col2.number_input(
    'Price (Max)', min_value=0, max_value=max_price, value=1300, step=50)
price_selected = [price_selected_0, price_selected_1]

# Filter for area
max_area = int(df_pararius['Living area'].max())
min_area = int(df_pararius['Living area'].min())
area_selected = st.sidebar.slider(
    'Total Area', min_area, max_area, (45, max_area))

# Filter for interior
my_expander = st.sidebar.expander(label='Advanced Filters')
with my_expander:
    interior_selected = st.multiselect("Interior", options=list(
        df_pararius['interior'].unique()), default=['Furnished'])
    status_selected = st.multiselect("Status", options=list(
        df_pararius['status'].unique()), default=list(df_pararius['status'].unique()))
    max_room = int(df_pararius['Rooms'].max())
    min_room = int(df_pararius['Rooms'].min())
    room_selected = st.slider(
        'Rooms', min_room, max_room, (min_room, max_room))


# apply filter
df_pararius = df_pararius[filter_]

# define order of best deal
df_pararius = df_pararius.sort_values(
    'deal', ascending=False).reset_index(drop=True)

# Here I will tell how many I want to check
max_val = int(df_pararius.index.max())
min_val = int(df_pararius.index.min())
index_selected = st.sidebar.slider(
    'Amout houses', min_val, max_val, (min_val, 10))

# apply filter
good = df_pararius[(df_pararius.index >= index_selected[0])
                   & (df_pararius.index <= index_selected[1])]

# MAP
pararius = folium.Map([52.0799838, 4.3113461],
                      zoom_start=12, tiles="cartodbdark_matter")

callback = ('function (row) {'
            "var marker = L.marker(new L.LatLng(row[0], row[1]), {color: 'blue'});"
            "var popup = L.popup({maxWidth: '300'});"
            "const display_text_price = {text: row[2]};"
            "const display_text_link_ = {text: row[3]};"
            "const area = {text: row[4]};"
            "const rooms = {text: row[5]};"
            "const garden = {text: row[6]};"
            "const index = {text: row[7]};"
            "const img = {text: row[8]};"

            "var mytext = $(`\
                            <div id='mytext' class='display_text' style='width: 100.0%; height: 100.0%;'>\
                                <img src=${img.text} title='titulo' width='150' height='100'/>\
                                <br>\
                                Index - ${index.text}<br>\
                                Price - € ${display_text_price.text}<br>\
                                Area - ${area.text} m² <br>\
                                Rooms - ${rooms.text}<br>\
                                Garden - ${garden.text} m² <br>\
                                <a href=https://www.pararius.com${display_text_link_.text} target='blank'> Source </a>\
                            </div>`)[0];"
            "popup.setContent(mytext);"
            "marker.bindPopup(popup);"
            'return marker};')
# prepare data to map
lats = good['latitude'].tolist()
lons = good['longitude'].tolist()
deal = good['deal'].tolist()
price = good['price'].tolist()
irl = good['irl'].tolist()
area = good['Living area'].tolist()
rooms = good['Rooms'].tolist()
garden = good['garden area'].tolist()
index = good.index.tolist()
image = good['image'].tolist()

locations = list(zip(lats, lons, price, irl, area,
                 rooms, garden, index, image))

# add data to map
FastMarkerCluster(data=locations, name='good', callback=callback,
                  show=True, tooltip='tooltip').add_to(pararius)
HeatMap(good[['latitude', 'longitude', 'deal']].values.tolist(),
        name='good HeatMap', show=False).add_to(pararius)
folium.LayerControl().add_to(pararius)

# plot map on streamlit
st.write(pararius)

# plot data on streamlit
good_ = good[['img', 'price', 'link', 'agency', 'Living area',
              'Rooms', 'garden area']].to_html(escape=False)

# prepare to download
towrite = io.BytesIO()
downloaded_file = good[["url", "price", "address", "street", "agency", "date"]].to_excel(
    towrite, encoding='utf-8', index=False, header=True)
towrite.seek(0)  # reset pointer
b64 = base64.b64encode(towrite.read()).decode()  # some strings
linko = f'<a href="data:application/vnd.openxmlformats-officedocument.spreadsheetml.sheet;base64,{b64}" download="rents_denhaag.xlsx">Download excel file</a>'
st.text(" \n")
st.markdown(linko, unsafe_allow_html=True)
st.text(" \n")
st.write(good_, unsafe_allow_html=True)
